## Place order

In [4]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import threading
import time

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
	
	def nextValidId(self, orderId: int):
		super().nextValidId(orderId)
		self.nextorderId = orderId
		print('The next valid order id is: ', self.nextorderId)

	def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
		print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
	def openOrder(self, orderId, contract, order, orderState):
		print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

	def execDetails(self, reqId, contract, execution):
		print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)




contract = Contract()
contract.symbol = "AAPL" # apple
contract.secType = "STK" # stk stand for stock
contract.currency = "USD" 
contract.exchange = "SMART"

app = IBapi()
app.connect('127.0.0.1', 7496, 123)

app.nextorderId = None

# print("Next Order ID: ", app.nextValidId(0))

def run_loop():
	app.run()

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

#Check if the API is connected via orderid
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		print()
		break
	else:
		print('waiting for connection')
		time.sleep(5)

#Create order object
order = Order()
order.action = 'SELL'
order.totalQuantity = 2
order.orderType = 'LMT'
order.lmtPrice = 194.26
order.eTradeOnly = False
order.firmQuoteOnly = False

#Place order
app.placeOrder(app.nextorderId, contract, order)
#app.nextorderId += 1

time.sleep(30)

#Cancel order 
print('cancelling order')
app.cancelOrder(app.nextorderId)

time.sleep(3)
app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


waiting for connection
The next valid order id is:  8


## Request Live Data

In [ ]:
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.ticktype import TickType, TickTypeEnum

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
    def nextValidId(self, orderId: int):
        
        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        self.reqMarketDataType(4)
        self.reqMktData(orderId, mycontract, "", 0, 0, [])
        
    def tickPrice(self, reqId, tickType, price, attrib):
        print(f"tickPrice. reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, price: {price}, attribs: {attrib}")
    # def tickSize(self, reqId, tickType, size):
    #     print(f"tickSize. reqId: {reqId}, tickType: {TickTypeEnum.to_str(tickType)}, size: {size}")
        
app = TestApp()
app.connect("127.0.0.1", 7496, 1000)

def run_loop():
	app.run()

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(3)
app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


tickPrice. reqId: 1, tickType: BID, price: 195.33, attribs: CanAutoExecute: 1, PastLimit: 0, PreOpen: 0
tickSize. reqId: 1, tickType: BID_SIZE, size: 7
tickPrice. reqId: 1, tickType: ASK, price: 195.35, attribs: CanAutoExecute: 1, PastLimit: 0, PreOpen: 0
tickSize. reqId: 1, tickType: ASK_SIZE, size: 1
tickPrice. reqId: 1, tickType: LAST, price: 195.34, attribs: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
tickSize. reqId: 1, tickType: LAST_SIZE, size: 1
tickSize. reqId: 1, tickType: BID_SIZE, size: 7
tickSize. reqId: 1, tickType: ASK_SIZE, size: 1
tickSize. reqId: 1, tickType: LAST_SIZE, size: 1
tickSize. reqId: 1, tickType: VOLUME, size: 147646
tickPrice. reqId: 1, tickType: HIGH, price: 196.15, attribs: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
tickPrice. reqId: 1, tickType: LOW, price: 194.03, attribs: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
tickPrice. reqId: 1, tickType: CLOSE, price: 193.22, attribs: CanAutoExecute: 0, PastLimit: 0, PreOpen: 0
tickPrice. reqId: 1, tickType: 

tickSize. reqId: 1, tickType: BID_SIZE, size: 3
tickSize. reqId: 1, tickType: BID_SIZE, size: 1
tickSize. reqId: 1, tickType: ASK_SIZE, size: 3


## Request Historical Market Data

In [ ]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
    def nextValidId(self, orderId: int):
        
        mycontract = Contract()
        mycontract.symbol = "AAPL"
        mycontract.secType = "STK"
        mycontract.exchange = "SMART"
        mycontract.currency = "USD"
        queryTime = (datetime.datetime.today() - datetime.timedelta(days=180)).strftime("%Y%m%d-%H:%M:%S")
        self.reqHistoricalData(orderId, mycontract, queryTime, "1 D", "1 min", "TRADES", 0, 1, 0, [])
    def historicalData(self, reqId, bar):
        print(f"Historical Data: {bar}")
    def historicalDataEnd(self, reqId, start, end):
        print(f"End of HistoricalData")
        print(f"Start: {start}, End: {end}")
app = TestApp()
app.connect("127.0.0.1", 7496, 1000)
app.run()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


ERROR 1 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 12.55 to 12


Historical Data: Date: 20230127  17:00:00, Open: 143.480000, High: 143.480000, Low: 143.220000, Close: 143.440000, Volume: 12, Average: 143.448000, BarCount: 8
Historical Data: Date: 20230127  17:01:00, Open: 143.330000, High: 143.330000, Low: 143.300000, Close: 143.300000, Volume: 9, Average: 143.324000, BarCount: 2
Historical Data: Date: 20230127  17:02:00, Open: 143.300000, High: 143.300000, Low: 143.300000, Close: 143.300000, Volume: 0, Average: 143.300000, BarCount: 0
Historical Data: Date: 20230127  17:03:00, Open: 143.340000, High: 143.340000, Low: 143.330000, Close: 143.330000, Volume: 5, Average: 143.338000, BarCount: 4
Historical Data: Date: 20230127  17:04:00, Open: 143.330000, High: 143.330000, Low: 143.330000, Close: 143.330000, Volume: 0, Average: 143.330000, BarCount: 0
Historical Data: Date: 20230127  17:05:00, Open: 143.330000, High: 143.330000, Low: 143.320000, Close: 143.330000, Volume: 4, Average: 143.326000, BarCount: 3
Historical Data: Date: 20230127  17:06:00, Op

## Download Historical Data

In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time
from datetime import datetime
import numpy as np

def download_data_from_ib(stock, start_date="01-01-2021", end_date = datetime.now(), size ='1 day'):

	class IBapi(EWrapper, EClient):
		def __init__(self):
			EClient.__init__(self, self)
			self.data = [] #Initialize variable to store candle

		def historicalData(self, reqId, bar):
			# print(f'Time: {bar.date} Close: {bar.close}')
			self.data.append([bar.date, bar.close])
			
	def run_loop():
		app.run()

	app = IBapi()
	app.connect('127.0.0.1', 7496, 123)

	#Start the socket in a thread
	api_thread = threading.Thread(target=run_loop, daemon=True)
	api_thread.start()

	time.sleep(1) #Sleep interval to allow time for connection to server

	#Create contract object
	eurusd_contract = Contract()
	eurusd_contract.symbol = stock
	eurusd_contract.secType = 'STK'
	eurusd_contract.exchange = 'SMART'
	eurusd_contract.currency = 'USD'

	start_date = datetime.strptime(start_date, '%m-%d-%Y')
	end_date = datetime.now()

	days = (end_date - start_date).days
	if days < 365:
		duration = "%d D" % days
	else:
		duration = "%d Y" % np.ceil(days / 365)
	# when getting daily data, look at regular trading hours only
	# to get accurate daily closing prices
	useRTH = True
	# round up current time to midnight for even days
	end_date = end_date.replace(
		hour=0, minute=0, second=0, microsecond=0
	)


	end_date = end_date.strftime(f"%Y%m%d %H:%M:%S UTC")
	# duration = '2 D'
	#Request historical candles

	print("start_date",start_date)
	print("end_date", end_date)
	print("duration", duration)
	app.reqHistoricalData(1, eurusd_contract, end_date, duration, size, 'BID', useRTH, 1, False, [])

	time.sleep(5) #sleep to allow enough time for data to be returned

	# #Working with Pandas DataFrames
	import pandas

	df = pandas.DataFrame(app.data, columns=['DateTime', 'Close'])
	# print(df)
	df['DateTime'] = pandas.to_datetime(df['DateTime'],
               format='%Y%m%d') 
	df.to_csv('data/' + stock + '.csv')  



	app.disconnect()

In [2]:
stocks = ["AAPL", "TSLA"]
for stock in stocks:
    download_data_from_ib(stock)


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


start_date 2021-01-01 00:00:00
end_date 20230807 00:00:00 UTC
duration 3 Y


ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


start_date 2021-01-01 00:00:00
end_date 20230807 00:00:00 UTC
duration 3 Y
